<a href="https://colab.research.google.com/github/DamnCrab/Real-ESRGAN-Inference-Demo/blob/main/Real_ESRGAN_Inference_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Real-ESRGAN Inference Demo

[![arXiv](https://img.shields.io/badge/arXiv-Paper-<COLOR>.svg)](https://arxiv.org/abs/2107.10833)
[![GitHub Stars](https://img.shields.io/github/stars/xinntao/Real-ESRGAN?style=social)](https://github.com/xinntao/Real-ESRGAN)
[![download](https://img.shields.io/github/downloads/xinntao/Real-ESRGAN/total.svg)](https://github.com/xinntao/Real-ESRGAN/releases)

This is a **Practical Image Restoration Demo** of our paper [''Real-ESRGAN: Training Real-World Blind Super-Resolution with Pure Synthetic Data''](https://arxiv.org/abs/2107.10833).
We extend the powerful ESRGAN to a practical restoration application (namely, Real-ESRGAN), which is trained with pure synthetic data. <br>
The following figure shows some real-life examples.

<img src="https://raw.githubusercontent.com/xinntao/Real-ESRGAN/master/assets/teaser.jpg" width="100%">

We provide a pretrained model (*RealESRGAN_x4plus.pth*) with upsampling X4.<br>
**Note that RealESRGAN may still fail in some cases as the real-world degradations are really too complex.**<br>
Moreover, it **may not** perform well on **human faces, text**, *etc*, which will be optimized later.
<br>

You can also find a **Portable Windows/Linux/MacOS executable files for Intel/AMD/Nvidia GPU.** in our [GitHub repo](https://github.com/xinntao/Real-ESRGAN). <br>
This executable file is **portable** and includes all the binaries and models required. No CUDA or PyTorch environment is needed.<br>
This executable file is based on the wonderful [Tencent/ncnn](https://github.com/Tencent/ncnn) and [realsr-ncnn-vulkan](https://github.com/nihui/realsr-ncnn-vulkan) by [nihui](https://github.com/nihui).

# 1. Preparations
Before start, make sure that you choose
* Runtime Type = Python 3
* Hardware Accelerator = GPU

in the **Runtime** menu -> **Change runtime type**

Then, we clone the repository, set up the envrironment, and download the pre-trained model.

In [ ]:
#检测使用的显卡
! /opt/bin/nvidia-smi
!pip install wandb
import wandb
wandb.init()

In [2]:
# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
# !pip install -r requirements.txt
# Download the pre-trained model
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 138 (delta 60), reused 104 (delta 35), pack-reused 0
Receiving objects: 100% (138/138), 2.07 MiB | 11.39 MiB/s, done.
Resolving deltas: 100% (60/60), done.
/content/Real-ESRGAN
     |████████████████████████████████| 131 kB 11.4 MB/s 
     |████████████████████████████████| 5.5 MB 41.6 MB/s 
     |████████████████████████████████| 185 kB 49.5 MB/s 
  Created wheel for basicsr: filename=basicsr-1.3.3.10-py3-none-any.whl size=163053 sha256=e1682ae7e8ffb2d92e905595c1438db8bdf080ab9e8fb549c81b704eb8ce6a0c
  Stored in directory: /root/.cache/pip/wheels/37/db/47/f3cf202930c96f8215354040c43f499325a6d621e5b587c03d
Successfully built basicsr
--2021-08-03 05:24:53--  https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth
Resolving github.com (github.com)... 140.82.121.3
Connecting to git

# 2. Upload Images

Upload the images to be processed by Real-ESRGAN

In [3]:
#网页上传方式
import os
from google.colab import files
import shutil

upload_folder = 'upload'
result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

# upload images
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)



Saving 序列 02_1.mp4 to 序列 02_1.mp4
move 序列 02_1.mp4 to upload/序列 02_1.mp4


In [4]:
#Drive上传方式
import os
from google.colab import files
import shutil

from google.colab import drive
drive.mount('/content/drive')
filename='[SHANA]videoplayback.mkv' #填文件名
shutil.move("/content/drive/{filename}", "/content/Real-ESRGAN/upload/")



Mounted at /content/drive


In [5]:
# 检测输入文件FPS。

import cv2
cap = cv2.VideoCapture(f'/content/upload/{filename}')

#fps = int(round(cap.get(cv2.CAP_PROP_FPS)))
fps = cap.get(cv2.CAP_PROP_FPS)
print(f'fps= {fps}')


fps= 0.0


In [ ]:
#ffmpeg -i in.mkv -codec copy -map 0 -f segment -segment_list out.csv -segment_frames 100,200,300,500,800 out%03d.nut

In [32]:
#更新ffmpeg 
!add-apt-repository ppa:jonathonf/ffmpeg-4 -y
!apt-get update
!apt-get install ffmpeg

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/jonathonf/ffmpeg-4/ubuntu bionic InRelease [15.9 kB]
Hit:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:10 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:14 h

In [33]:
shell ffmpeg -version

ffmpeg version 4.3.2-0york0~18.04 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
configuration: --prefix=/usr --extra-version='0york0~18.04' --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --e

In [47]:
!apt-get install nodejs-dev node-gyp libssl1.0-dev
!apt-get install npm
!$ npm install --global ffmpeg-progressbar-cli


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libarmadillo-dev libarpack2-dev libcurl4-gnutls-dev libdap-dev
  libdapserver7v5 libepsilon-dev libfreexl-dev libfyba-dev libgeos-dev
  libgeotiff-dev libgif-dev libhdf4-alt-dev libjson-c-dev libkml-dev
  libkmlconvenience1 libkmlregionator1 libkmlxsd1 libminizip-dev libnetcdf-dev
  libogdi3.2-dev libopenjp2-7-dev libpoppler-dev libpoppler-private-dev
  libpq-dev libproj-dev libqhull-dev libqhull-r7 libspatialite-dev
  libsqlite3-dev libsuperlu-dev liburiparser-dev libwebp-dev libxerces-c-dev
  unixodbc-dev
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  curl gyp javascript-common libc-ares2 libcurl3-gnutls libcurl4
  libcurl4-gnutls-dev libjs-async libjs-inherits libjs-node-uuid
  libmysqlclient20 libuv1-dev node-abbrev node-ansi node-async
  node-balanced-match node

In [48]:
! npm install --global ffmpeg-progressbar-cli









WARN engine ffmpeg-progressbar-cli@1.5.0: wanted: {"node":">=8.11.3"} (current: {"node":"8.10.0","npm":"3.5.2"})
WARN engine ffmpeg-progressbar-cli@1.5.0: wanted: {"node":">=8.11.3"} (current: {"node":"8.10.0","npm":"3.5.2"})































































/usr/local/bin/ffmpeg-bar -> /usr/local/lib/node_modules/ffmpeg-progressbar-cli/lib/main.js



/usr/local/lib
└─┬ ffmpeg-progressbar-cli@1.5.0 
  ├─┬ @sidneys/cli-progress@2.2.0 
  │ └── colors@1.4.0 
  ├── app-root-path@2.2.1 
  ├─┬ chalk@2.4.2 
  │ ├─┬ ansi-styles@3.2.1 
  │ │ └─┬ color-convert@1.9.3 
  │ │   └── color-name@1.1.3 
  │ ├── escape-string-regexp@1.0.5 
  │ └─┬ supports-color@5.5.0 
  │   └── has-flag@3.0.0 
  ├── ellipsize@0.1.0 
  ├── ini@1.3.8 
  ├── moment@2.29.1 
  ├── moment-duration-format@2.3.2 
  ├─┬ string-width@2.1.1 
  │ ├── is-fullwidth-code-point@2.0.0 
  │ └─┬ strip-ansi@4.0.0 
  │   └── ansi-regex@3.0.0 
  ├─┬ which@1.3.1 
  │ └── isexe@2.0.0 
  └─┬ window-size@1.

In [ ]:
#拆帧
filename='[SHANA]videoplayback.mkv'
%shell rm -rf '/content/Real-ESRGAN/frame'
%shell mkdir -p '/content/Real-ESRGAN/frame'
%shell ffmpeg -hwaccel_device 0 -i '/content/Real-ESRGAN/upload/{filename}' '/content/Real-ESRGAN/frame/%08d.png'


ffmpeg version 4.3.2-0york0~18.04 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version='0york0~18.04' --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab

# 3. Inference


In [8]:
#超分
# if it is out of memory, try to use the `--tile` option 爆显存调tile 现值100
!python inference_realesrgan.py --model_path experiments/pretrained_models/RealESRGAN_x4plus.pth --input frame --scale 4 --half --tile 100

	Tile 5/220
	Tile 6/220
	Tile 7/220
	Tile 8/220
	Tile 9/220
	Tile 10/220
	Tile 11/220
	Tile 12/220
	Tile 13/220
	Tile 14/220
	Tile 15/220
	Tile 16/220
	Tile 17/220
	Tile 18/220
	Tile 19/220
	Tile 20/220
	Tile 21/220
	Tile 22/220
	Tile 23/220
	Tile 24/220
	Tile 25/220
	Tile 26/220
	Tile 27/220
	Tile 28/220
	Tile 29/220
	Tile 30/220
	Tile 31/220
	Tile 32/220
	Tile 33/220
	Tile 34/220
	Tile 35/220
	Tile 36/220
	Tile 37/220
	Tile 38/220
	Tile 39/220
	Tile 40/220
	Tile 41/220
	Tile 42/220
	Tile 43/220
	Tile 44/220
	Tile 45/220
	Tile 46/220
	Tile 47/220
	Tile 48/220
	Tile 49/220
	Tile 50/220
	Tile 51/220
	Tile 52/220
	Tile 53/220
	Tile 54/220
	Tile 55/220
	Tile 56/220
	Tile 57/220
	Tile 58/220
	Tile 59/220
	Tile 60/220
	Tile 61/220
	Tile 62/220
	Tile 63/220
	Tile 64/220
	Tile 65/220
	Tile 66/220
	Tile 67/220
	Tile 68/220
	Tile 69/220
	Tile 70/220
	Tile 71/220
	Tile 72/220
	Tile 73/220
	Tile 74/220
	Tile 75/220
	Tile 76/220
	Tile 77/220
	Tile 78/220
	Tile 79/220
	Tile 80/220
	Tile 81/220
	Til

# 5. Download Results


In [ ]:
# Download the results
zip_filename = 'Real-ESRGAN_result.zip'
if os.path.exists(zip_filename):
  os.remove(zip_filename)
os.system(f"zip -r -j {zip_filename} results/*")
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>